In [1]:
import numpy as np
import pandas as pd
import astropy
import ccdproc
from ccdproc import CCDData, Combiner
from astropy import units as u
from astropy.visualization import SqrtStretch
import matplotlib.pyplot as plt 
from matplotlib.colors import LogNorm
from photutils import centroid_com, centroid_1dg, centroid_2dg
from photutils import CircularAperture
from photutils import aperture_photometry
from photutils import Background2D
from photutils import MedianBackground
from scipy.ndimage import shift
import gc                                                           
gc.enable()

C:\Users\Msi Raider\AppData\Local\Temp\ipykernel_13172\1502218167.py:10: DeprecationWarning: `photutils.centroid_com` is a deprecated alias for `photutils.centroids.centroid_com`. Instead, please use `from photutils.centroids import centroid_com` to silence this warning.
  from photutils import centroid_com, centroid_1dg, centroid_2dg
C:\Users\Msi Raider\AppData\Local\Temp\ipykernel_13172\1502218167.py:10: DeprecationWarning: `photutils.centroid_1dg` is a deprecated alias for `photutils.centroids.centroid_1dg`. Instead, please use `from photutils.centroids import centroid_1dg` to silence this warning.
  from photutils import centroid_com, centroid_1dg, centroid_2dg
C:\Users\Msi Raider\AppData\Local\Temp\ipykernel_13172\1502218167.py:10: DeprecationWarning: `photutils.centroid_2dg` is a deprecated alias for `photutils.centroids.centroid_2dg`. Instead, please use `from photutils.centroids import centroid_2dg` to silence this warning.
  from photutils import centroid_com, centroid_1dg, ce

First we load our images into the images variable using the ImageFileCollection function, abd we print which files have been loaded in to manually check the function has performed as expected

In [2]:
images = ccdproc.ImageFileCollection(".") #loading in image files
print(dir(images))
print(images.files) #Printing the loaded files

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_all_keywords', '_dict_from_fits_header', '_ext', '_filenames', '_files', '_find_fits_by_reading', '_find_keywords_by_values', '_fits_files_in_directory', '_fits_summary', '_generator', '_get_files', '_glob_exclude', '_glob_include', '_location', '_paths', '_set_column_name_case_to_match_keywords', '_summary', 'ccds', 'data', 'ext', 'files', 'files_filtered', 'filter', 'glob_exclude', 'glob_include', 'hdus', 'headers', 'keywords', 'location', 'refresh', 'sort', 'summary', 'values']
['NGC 2323_B_10.000secs_00002235.fit', 'NGC 2323_B_10.000secs_00002236.fit', 'NGC 2323_B_10.000secs_00002237.fit', 'NGC 2323_B_10.000secs_00002238.fit', 'NGC 2323_B_10

Each band is then loaded into their respective variables and each images name is printed for manually checking that the collections only contain their respective bands.

In [3]:
# Reading in the unprocessed scientific images

imagesV = ccdproc.ImageFileCollection(".",glob_include='NGC 2323_V_*')
for fnV in imagesV.files_filtered(PICTTYPE = 1): #printing image file names
    print(fnV)
imagesB = ccdproc.ImageFileCollection(".",glob_include='NGC 2323_B_*')
for fnB in imagesB.files_filtered(PICTTYPE = 1): #printing image file names
    print(fnB)
imagesR = ccdproc.ImageFileCollection(".",glob_include='NGC 2323_R_*')
for fnR in imagesR.files_filtered(PICTTYPE = 1): #printing image file names
    print(fnR)

NGC 2323_V_10.000secs_00002245.fit
NGC 2323_V_10.000secs_00002246.fit
NGC 2323_V_10.000secs_00002247.fit
NGC 2323_V_10.000secs_00002248.fit
NGC 2323_V_10.000secs_00002249.fit
NGC 2323_V_10.000secs_00002250.fit
NGC 2323_V_10.000secs_00002251.fit
NGC 2323_V_10.000secs_00002252.fit
NGC 2323_V_10.000secs_00002253.fit
NGC 2323_V_10.000secs_00002254.fit
NGC 2323_V_120.000secs_00002270.fit
NGC 2323_V_120.000secs_00002271.fit
NGC 2323_V_120.000secs_00002272.fit
NGC 2323_V_120.000secs_00002273.fit
NGC 2323_V_120.000secs_00002274.fit
NGC 2323_B_10.000secs_00002235.fit
NGC 2323_B_10.000secs_00002236.fit
NGC 2323_B_10.000secs_00002237.fit
NGC 2323_B_10.000secs_00002238.fit
NGC 2323_B_10.000secs_00002239.fit
NGC 2323_B_10.000secs_00002240.fit
NGC 2323_B_10.000secs_00002241.fit
NGC 2323_B_10.000secs_00002242.fit
NGC 2323_B_10.000secs_00002243.fit
NGC 2323_B_10.000secs_00002244.fit
NGC 2323_B_120.000secs_00002265.fit
NGC 2323_B_120.000secs_00002266.fit
NGC 2323_B_120.000secs_00002267.fit
NGC 2323_B_1

In [4]:
V_im_0 = CCDData.read("NGC 2323_V_10.000secs_00002245.fit", unit = "adu")
V_im_1 = CCDData.read("NGC 2323_V_10.000secs_00002246.fit", unit = "adu")

print(V_im_0)
print(V_im_1)

Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'.


[[2385 2346 2428 ... 2335 2267 2317]
 [2331 2326 2348 ... 2261 2243 2266]
 [2350 2448 2347 ... 2360 2188 2292]
 ...
 [2394 2285 2379 ... 2328 2359 2327]
 [2328 2377 2333 ... 2360 2321 2232]
 [2378 2291 2335 ... 2251 2319 2287]]
[[2342 2296 2323 ... 2321 2252 2303]
 [2321 2396 2376 ... 2370 2325 2257]
 [2485 2333 2400 ... 2321 2351 2228]
 ...
 [2361 2261 2221 ... 2285 2383 2290]
 [2318 2388 2318 ... 2353 2251 2285]
 [2359 2320 2340 ... 2284 2264 2351]]


# Image Reduction
<br>
To reduce our images so that the background and noise has minimal effect on our analysis of NGC 2323, we follow the steps from section 4.5.3 of Rieke:<br>

1. data' = data - bias'<br>
2. dark'= dark - bias'<br>
3. flat' = flats - bias'<br>
4. data'' = data' - $\alpha $dark'<br>
5. flat'' = flat' - $\beta $dark'<br>
6. proc_image = data''/flat''<br>

<br>
We first load in all our images that we are going to use for our reductions

In [7]:
# Reading in the darks. biases and flats

bias_median = CCDData.read("bias_median.fits", unit = "adu")
dark_median = CCDData.read("dark_median.fits", unit = "adu")
flats_median_V = CCDData.read("Flat_V_median.fits", unit = "adu")
flats_median_B = CCDData.read("Flat_B_median.fits", unit = "adu")
flats_median_R = CCDData.read("Flat_R_median.fits", unit = "adu")


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu2 in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu2 in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu2 in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu2 in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu2 in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu2 in the FITS file. [astropy.nddata.ccddata]


We then read the images data and load them into their respective lists using the read function.

In [8]:
# Reading in science images

imV = [CCDData.read(fnV, unit = "adu") for fnV in imagesV.files_filtered(PICTTYPE = 1)]
imB = [CCDData.read(fnB, unit = "adu") for fnB in imagesB.files_filtered(PICTTYPE = 1)]
imR = [CCDData.read(fnR, unit = "adu") for fnR in imagesR.files_filtered(PICTTYPE = 1)]

Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'.
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'.
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'.


To check the images have been read properly we print the first two images.

In [9]:
print(imV[0])
print('\n')
print(imV[1])

[[2385 2346 2428 ... 2335 2267 2317]
 [2331 2326 2348 ... 2261 2243 2266]
 [2350 2448 2347 ... 2360 2188 2292]
 ...
 [2394 2285 2379 ... 2328 2359 2327]
 [2328 2377 2333 ... 2360 2321 2232]
 [2378 2291 2335 ... 2251 2319 2287]]


[[2342 2296 2323 ... 2321 2252 2303]
 [2321 2396 2376 ... 2370 2325 2257]
 [2485 2333 2400 ... 2321 2351 2228]
 ...
 [2361 2261 2221 ... 2285 2383 2290]
 [2318 2388 2318 ... 2353 2251 2285]
 [2359 2320 2340 ... 2284 2264 2351]]


A function was then created to reduce each band respectively. The input of the funtion requires the collection variable, the image data and the flats each of the respective images band which we are reducing.<br>

The fuction takes the iterates through the each images data and subtracts the bias_median from each image. Then the darks are subtraced from the images, within this step, the dark image is scaled so the exposure time is equivilent to the exposure time of the respective image. Finally, the image is flat corrected using the median_flat file for the respective band.<br>

The reduced image is then saved as a fit file with the word 'proc' at the start to indicate the image has been processed and reduced

In [10]:
def reduction(collection: ccdproc.ImageFileCollection, image: list, flat: list) -> None:
    
    #Median subtraction
    for idx, thisimage in enumerate(image):
        image[idx] = ccdproc.subtract_bias(thisimage, bias_median)
    
    for idx, thisimage in enumerate(image):
        image[idx] = ccdproc.subtract_dark(thisimage, dark_median, exposure_time = 'EXPTIME', exposure_unit = u.second, scale = True)
        
    for idx, thisimage in enumerate(image):
        image[idx] = ccdproc.flat_correct(thisimage, flat)

    newname = []

    for fn in collection.files_filtered(PICTTYPE = 1):
        newname.extend(["proc_" + fn])
    print(newname)

    for idx, thisimage in enumerate(image):
        tempimages = [thisimage]
        temp = ccdproc.Combiner(tempimages,dtype=np.float32).median_combine()
        temp.meta = thisimage.meta
        temp.write(newname[idx])


Carrying out the reduction for the V, B and R bands. This section take a little bit to run.

In [11]:
reduction(imagesV, imV, flats_median_V)
reduction(imagesB, imB, flats_median_B)
reduction(imagesR, imR, flats_median_R)

['proc_NGC 2323_V_10.000secs_00002245.fit', 'proc_NGC 2323_V_10.000secs_00002246.fit', 'proc_NGC 2323_V_10.000secs_00002247.fit', 'proc_NGC 2323_V_10.000secs_00002248.fit', 'proc_NGC 2323_V_10.000secs_00002249.fit', 'proc_NGC 2323_V_10.000secs_00002250.fit', 'proc_NGC 2323_V_10.000secs_00002251.fit', 'proc_NGC 2323_V_10.000secs_00002252.fit', 'proc_NGC 2323_V_10.000secs_00002253.fit', 'proc_NGC 2323_V_10.000secs_00002254.fit', 'proc_NGC 2323_V_120.000secs_00002270.fit', 'proc_NGC 2323_V_120.000secs_00002271.fit', 'proc_NGC 2323_V_120.000secs_00002272.fit', 'proc_NGC 2323_V_120.000secs_00002273.fit', 'proc_NGC 2323_V_120.000secs_00002274.fit']


To check the reduction was completed correctly, we read two different images into variables and then print them to make sure there are no outstanding issues

In [14]:
V_im_0 = CCDData.read("proc_NGC 2323_V_10.000secs_00002245.fit", unit = "adu")
V_im_1 = CCDData.read("proc_NGC 2323_V_10.000secs_00002246.fit", unit = "adu")
print("proc_NGC 2323_V_10.000secs_00002245.fit")
print(V_im_0)
print("\n**proc_NGC 2323_V_10.000secs_00002246.fit**")
print(V_im_1)

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to  2879792.379 from OBSGEO-[LBH].
Set OBSGEO-Z to -3897419.410 from OBSGEO-[LBH]'.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
proc_NGC 2323_V_10.000secs_00002245.fit
[[235.59409 196.1842  296.4103  ... 251.64024 173.20647 227.73346]
 [172.72185 181.05339 212.79431 ... 169.74855 161.13628 197.65532]
 [189.33896 308.5732  205.83232 ... 280.56808  97.06361 199.22028]
 ...
 [312.0172  165.47163 319.36325 ... 224.86752 261.40912 234.56325]
 [244.14525 317.2171  272.14932 ... 262.836   229.82867 118.58276]
 [312.8529  209.13023 283.4374  ... 154.49707 220.79951 188.45284]]

**proc_NGC 2323_V_10.000secs_00002246.fit**
[[192.10654  145.48515  189.44179  ... 236.83044  156.43434  211.56326 ]
 [162.70503  251.87837  241.27936  ... 284.44885  253.37753  187.17513 ]
 [325.18604  193.85529  260.0103   ... 238.62448  282.15393  126.17184 ]
 ...
 [264.0785   131.38672   94.562256 ... 178.4217